In [1]:
#%conda list
#pip install git+https://github.com/TomSchimansky/CustomTkinter.git

# Imports

In [2]:
%matplotlib tk

from customtkinter import *
import tkinter
from PIL import ImageTk, Image
import itertools
import serial
import time
import pandas as pd
import numpy as np
import sys
import glob
import serial
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.animation as animation
from tkinter import filedialog
import os
from datetime import datetime
from matplotlib.widgets import Slider

In [3]:
def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

In [4]:
def is_clicked(record_protocol, test_window, label):
    exp_data = pd.read_csv('/Users/armenmkrtumyan/Desktop/oqni/DataLogger_STM/protocols/'+ record_protocol.replace('\\','')+'.csv')
    label.destroy()
    time_var = StringVar(value = exp_data['time_interval'].sum())
    time_interval = CTkLabel(test_window, textvariable =  time_var, font=('Helvetica bold', 26),
                            width = 110, height = 25)
    time_interval.grid(row=4,column=1)
#     myLabel = Label(test_window, text = '000000', bg="white", fg="white")
#     myLabel.config(font=('Helvetica bold', 26))
#     myLabel.grid(row=4,column=1)
#     myLabel = Label(test_window, text = exp_data['time_interval'].sum(), bg="white")
#     myLabel.config(font=('Helvetica bold', 26))
#     myLabel.grid(row=4,column=1)
    
def testing():
    test_window = CTkToplevel()
    test_window.title("Test Run")
    
    default_path = '/Users/armenmkrtumyan/Desktop/oqni/DataLogger_STM/recordings/'
    protocols = [] #for stering Exp names

    #Getting protocols from folder and storing them
    with os.scandir('/Users/armenmkrtumyan/Desktop/oqni/DataLogger_STM/protocols') as entries:
        for entry in entries:
            if(entry.name[0] == 'E'):
                protocols.append((entry.name)[:-4])
    protocols.sort()

    # setting the windows size
    test_window.geometry("670x150+0+480")

    # declaring string variable for storing Entries
    database_path= StringVar(test_window)
    recording_folder= StringVar(test_window)
    subject_number= StringVar(test_window)
    date_yymmdd= StringVar(test_window)
    time_s= StringVar(test_window)
    OPT_placement = StringVar(test_window)
    IMU_placement = StringVar(test_window)
    record_protocol = StringVar(test_window)
    limb = StringVar(test_window)

    #setting entries
    database_path.set(default_path)
    recording_folder.set("BL-001")
    subject_number.set("000")
    date_yymmdd.set("000000")
    time_s.set("1")
    OPT_placement.set("1.1, 1.2")
    opt_dropdown = CTkOptionMenu(test_window, variable = OPT_placement, values = ["1.1, 1.2", "2.1, 2.2", "3.1, 3.2"],
                                 button_hover_color = "green", anchor = 'center', width = 100, height = 20, 
                                 dynamic_resizing = True)
    IMU_placement.set("above knee")
    imu_dropdown = CTkOptionMenu(test_window, variable = IMU_placement, values = ["above knee", "below knee"], 
                                 button_hover_color = "green", anchor = 'center', width = 100, height = 20, 
                                 dynamic_resizing = True)
    
    limb.set("left leg")
    limb_dropdown = CTkOptionMenu(test_window, variable = limb, values = ["left leg", "right leg"], 
                                  button_hover_color = "green", anchor = 'center', width = 100, height = 20, 
                                  dynamic_resizing = True)

    #Creating labels and entries
    database_label = CTkLabel(test_window, text = 'Database Path', font=('calibre',10, 'bold'), anchor=SW, justify=LEFT)
    database_entry = CTkEntry(test_window, width=440, height = 25, textvariable = database_path, 
                              font=('calibre',10,'normal'), corner_radius = 10)
    recording_label = CTkLabel(test_window, text = 'Recording folder', font = ('calibre',10,'bold'), anchor=SW, justify=LEFT)
    recording_entry= CTkEntry(test_window, width = 110, height = 25, textvariable = recording_folder, 
                              font = ('calibre',10,'normal'), corner_radius = 10, border_width = 3)
    subject_entry = CTkEntry(test_window, width = 110, height = 25,  font = ('calibre',10,'bold'), 
                             corner_radius = 10, textvariable = subject_number)
    date_entry = CTkEntry(test_window, width = 110, height = 25, textvariable = date_yymmdd, 
                          font = ('calibre',10,'bold'), corner_radius = 10)
    time_entry = CTkEntry(test_window, width = 110, height = 25, textvariable = time_s, font = ('calibre',10,'bold'),
                         corner_radius = 10)
    OPT_label = CTkLabel(test_window, text = 'OPT Placement:', font=('calibre',10, 'bold'))
    IMU_label = CTkLabel(test_window, text = "IMU Placement:", font=('calibre',10, 'bold'))
    protocol_label = CTkLabel(test_window,text = "Recording Protocol:", font=('calibre',10, 'bold'))
    limb_label = CTkLabel(test_window,text = "Limb:", font=('calibre',10, 'bold'))

    record_protocol.set("Exp\#1")
    protocol_dropdown = CTkOptionMenu(test_window, variable = record_protocol, values = protocols, 
                                      command=lambda record_protocol: is_clicked(record_protocol, test_window, time_interval), 
                                      button_hover_color = "green", anchor = 'center', width = 100, height = 20, 
                                      dynamic_resizing = True)
    protocol_length_label = CTkLabel(test_window, text = "Protocol Length:", font=('calibre',10, 'bold'))

    
    #Label for showing overall seconds of a specific protocol
    exp_data = pd.read_csv('/Users/armenmkrtumyan/Desktop/oqni/DataLogger_STM/protocols/'+ (record_protocol.get()).replace('\\','')+'.csv')
    time_var = StringVar(value = exp_data['time_interval'].sum())
    time_interval = CTkLabel(test_window, textvariable =  time_var, font=('Helvetica bold', 26), 
                             width = 110, height = 25)
    print(time_interval)
    #Creating buttons
    start_button=CTkButton(test_window,text = 'Start', command = lambda: start(database_path,recording_folder,
                                                                               subject_number,date_yymmdd,time_s,
                                                                               OPT_placement,
                                                                               IMU_placement,record_protocol,limb,
                                                                               test_window),
                          hover_color = "green", width = 110, height = 25)
    browsebutton = CTkButton(test_window, text="Browse", command=lambda: browsefunc(database_path),
                            hover_color = "green", width = 110, height = 25)
    stop_button = CTkButton(test_window, text="Stop", command = stop,hover_color = "green",width = 110, height = 25)

    root.update() #Did not update entries otherwise

    #Putting everything on a grid
    database_label.grid(row=0,column=0)
    database_entry.grid(row=0, column=1, columnspan=4)
    browsebutton.grid(row=0,column=5)
    recording_label.grid(row=1,column=0)
    recording_entry.grid(row=1,column=1)
    subject_entry.grid(row=1,column=2)
    date_entry.grid(row=1,column=3)
    time_entry.grid(row=1,column=4)
    OPT_label.grid(row=2, column=0)
    opt_dropdown.grid(row=2, column=1)
    IMU_label.grid(row=2,column=2)
    imu_dropdown.grid(row=2,column=3)
    protocol_label.grid(row=3,column=0)
    protocol_dropdown.grid(row=3,column=1)
    limb_label.grid(row=3, column=2)
    limb_dropdown.grid(row=3, column=3)
    start_button.grid(row=3,column=5)
    protocol_length_label.grid(row=4,column=0)
    time_interval.grid(row=4,column=1)
    stop_button.grid(row=4,column=5)

    test_window.mainloop()

In [5]:
def info_clicked_2(value, root):
#     myLabel = Label(root, text = '000000000', bg="white", fg="white")
#     myLabel.config(font=('Helvetica bold', 26))
#     myLabel.grid(row=4,column=2)
#     myLabel = Label(root, text = value, bg="white")
#     myLabel.config(font=('Helvetica bold', 26))
    myLabel = CTkLabel(root, text = value)
    myLabel.grid(row=4,column=2)
def stop():
    global stopped
    stopped = 1
def browsefunc(database_path):
    filename = filedialog.askdirectory()
    database_path.set(filename)
def resizer(event):
    image_file = Image.open('/Users/armenmkrtumyan/Desktop/oqni/DataLogger_STM/protocols/pics/'+str(exp_data['label_index'][i]) +'.png')
    image_resized = image_file.resize((event.width, event.length), Image.LANCZOS)
    image_tk = ImageTk.PhotoImage(image_resized)


In [6]:
def start(database_path,recording_folder,subject_number,date_yymmdd,time_s,OPT_placement,IMU_placement,record_protocol,limb, test_window):
    data_top = CTkToplevel()
    data_top.geometry('600x600+900+0')
    data_top.title("Test Run")
    #data_top.bind('<Configure>', resizer)
    #This variable is used for stopping the testing process
    global stopped
    stopped = 0
    
    #As soon as the start is clicked, the dates are being set
    date_yymmdd.set(datetime.today().strftime('%Y-%m-%d'))
    time_s.set(datetime.today().strftime('%H:%M:%S'))
    
    #Getting the user inputs
    path = database_path.get()
    folder = recording_folder.get()
    subject = subject_number.get()
    date = date_yymmdd.get()
    my_time = time_s.get()
    opt = OPT_placement.get()
    imu = IMU_placement.get()
    protocol = record_protocol.get()
    lim = limb.get()
    
    #getting the data of specific protocol
    exp_data = pd.read_csv('/Users/armenmkrtumyan/Desktop/oqni/DataLogger_STM/protocols/'+protocol.replace('\\','')+'.csv')
    overall_time = exp_data['time_interval'].sum()
    max_time = exp_data['time_interval'].sum()
    
    for i in range(len(exp_data)): #Looping through images
        frame = CTkFrame(data_top, width=600, height=600)
        frame.pack()
        frame.place(anchor='center', relx=0.5, rely=0.5)
        
        image_file = Image.open('/Users/armenmkrtumyan/Desktop/oqni/DataLogger_STM/protocols/pics/'+str(exp_data['label_index'][i]) +'.png')
        image_resized = image_file.resize((600, 600))
        image_tk = ImageTk.PhotoImage(image_resized)
        
#        image_tk = CTkImage(light_image=Image.open('/Users/armenmkrtumyan/Desktop/oqni/DataLogger_STM/protocols/pics/'+str(exp_data['label_index'][i]) +'.png'),
#                                  size=(600, 600))
#        leg_image = PhotoImage(file='/Users/armenmkrtumyan/Desktop/oqni/DataLogger_STM/protocols/pics/'+str(exp_data['label_index'][i]) +'.png')
        
#         picture_canvas = tkinter.Canvas(root, width=500, height = 500)
#         picture_canvas.pack(fill="both", expand=True)
#         picture_canvas.create_image(0,0,image=bg,anchor="nw")
#         picture_canvas.image=bg
    
        my_label = CTkLabel(frame, image=image_tk, text ="")
        my_label['image'] = image_tk
        my_label.pack(fill=BOTH, expand=YES)
        k = exp_data['time_interval'][i]
        while(k > 0 and stopped != 1):
            info_clicked_2(str(overall_time) + "/" + str(max_time) + ", " + str(exp_data['labels'][i]) + " " + str(k),
                                                                                test_window)
            #info_clicked_2(str(overall_time) + ":" + str(k) + " " + str(exp_data['labels'][i]), test_window)
            data_top.update()
            time.sleep(1)
            k-=1
            overall_time-=1
            root.update()  
        data_top.update()
        for widget in frame.winfo_children():
            widget.destroy()
        if(stopped == 1):
            for widget in frame.winfo_children():
                widget.destroy()
            break
    os.chdir(str(path))
    if ((os.path.exists(str(folder)+'/')) and (stopped != 1)):
        os.chdir(str(folder)+'/')
        exp_data.to_csv(os.getcwd()+ "/" + str(date) + "-" + str(my_time).replace(':', '-') + '.csv')
    else:
        if(stopped != 1):
            os.mkdir(str(folder)+'/')
        else:
            if(not os.path.exists('BL-000/')):
                os.mkdir('BL-000/')
            folder = 'BL-000'
        os.chdir(str(folder)+'/')
        exp_data.to_csv(os.getcwd()+ "/" + str(date) + "-" + str(my_time).replace(':', '-') + '.csv')
    info_clicked_2('', test_window)
    data_top.destroy()
    stopped = 0

### Live plotting

In [7]:
# def open_graph(port_name):
#     global ani
#     # Parameters
#     x_len = 200         # Number of points to display

#     # Create figure for plotting
#     fig, ax = plt.subplots(figsize=(9.3,6.6), dpi=150)
#     fig.canvas.manager.set_window_title('Sensor inputs')

#     xs = np.arange(x_len)
#     ys = np.zeros(x_len)
#     y2s = np.zeros(x_len)
#     line, = ax.plot(xs, ys)
#     line2, = ax.plot(xs, y2s)

#     # Add labels
#     ax.set_title('Sensor inputs over time')
#     ax.set_xlabel('Time')
#     ax.set_ylabel('Input')

#     ax_xmax = plt.axes([0.1, 0.01, 0.8, 0.03])
#     slider_xmax = Slider(ax_xmax, 'X max', 0, x_len, valinit=x_len)

#     def animate(i):
#         nonlocal x_len, ys, y2s, xs
        
#         if(x_len > ys.size):
#             print("YEP")
#         ys = ys[1:]
#         ys = np.append(ys, np.random.randint(70000, 100000))
#         #print(ys[int(-1 * x_len):])
        
#         y2s = y2s[1:]
#         y2s = np.append(y2s, np.random.randint(100000, 200000))

        
#         xs = xs[np.floor(-x_len).astype(int):]
#         ys = ys[np.floor(-x_len).astype(int):]
#         #line.set_xdata(xs)
#         #line2.set_xdata(xs)
#         y2s = y2s[np.floor(-x_len).astype(int):]
#         line.set_data(xs, ys)
#         line2.set_data(xs, y2s)

#         # Update the x-axis limit based on the slider value
#         if x_len != slider_xmax.val:
#             x_len = slider_xmax.val
#             ax.set_xlim([0, x_len])

#         ax.relim()
#         ax.autoscale(enable=True, axis='y')
#         return line, line2
#     ani = animation.FuncAnimation(fig, animate, interval=1, blit=True)
#     plt.show()

In [8]:
def open_graph(port_name):
    global ani
    # Parameters
    x_len = 200         # Number of points to display

    # Create figure for plotting
    fig, ax = plt.subplots(figsize=(9.3,6.6), dpi=150)
    fig.canvas.manager.set_window_title('Sensor inputs')

    xs = np.arange(x_len)
    ys = np.zeros(x_len)
    y2s = np.zeros(x_len)
    line, = ax.plot(xs, ys)
    line2, = ax.plot(xs, y2s)

    # Add labels
    ax.set_title('Sensor inputs over time')
    ax.set_xlabel('Time')
    ax.set_ylabel('Input')
    #ax.legend(loc='upper left')

    ax_xmax = plt.axes([0.1, 0.01, 0.8, 0.03])
    slider_xmax = Slider(ax_xmax, 'X max', 0, x_len, valinit=x_len/2)

    # This function is called periodically from FuncAnimation
    def animate(i):
        nonlocal ys,y2s
        def update(val):
            #ani.event_source.stop()  # stop the animation when slider is being changed
            xmax = slider_xmax.val
            ax.set_xlim([0, xmax])
            ani.event_source.start()  # restart the animation
            
                
        ys[:-1] = ys[1:]
        ys[-1] = np.random.randint(70000, 100000)
        
        y2s[:-1] = y2s[1:]
        y2s[-1] = np.random.randint(100000, 200000)

        line.set_ydata(ys)
        line2.set_ydata(y2s)
        ax.relim()
        ax.autoscale(enable=True, axis='y')
        slider_xmax.on_changed(update)
        return line, line2

    ani = animation.FuncAnimation(fig, animate, interval=1, blit=True)
    plt.show()


In [9]:
# def open_graph(port_name):
#     #global ani

#     # Parameters
#     x_len = 200         # Number of points to display

#     # Create figure for plotting
#     fig = plt.figure(figsize=(9.3,6.6), dpi=150)
#     fig.canvas.manager.set_window_title('Sesnor inputs')

#     ax = fig.add_subplot(1, 1, 1)
#     xs = list(range(0, 200))
#     ys = [0] * x_len
#     y2s = [0] * x_len

#     # Create a blank line. We will update the line in animate
#     #line, = ax.plot(xs, ys)
#     line, = ax.plot(xs, ys)
#     line2, = ax.plot(xs, y2s)

#     # Add labels
#     plt.title('Sensor Inputs over time')
#     plt.xlabel('Time')
#     plt.ylabel('Input')

#     ax_xmax = plt.axes([0.1, 0.01, 0.8, 0.03])
#     slider_xmax = Slider(ax_xmax, 'X max', 0, 200, valinit=200)

#     # This function is called periodically from FuncAnimation

#     #ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fancybox=True, shadow=True)

#     def animate(i):
#         nonlocal ys, y2s, ani
#         def update(val):
#             xmax = slider_xmax.val
#             ax.set_xlim([0, xmax])
                
#         temp_c = np.random.randint(100000,200000)
#         temp_2c = np.random.randint(70000,100000)

#         # Add y to list
#         ys.append(temp_c)
#         y2s.append(temp_2c)
        
#         # Limit y list to set number of items
#         ys = ys[-x_len:]
#         y2s = y2s[-x_len:]
        
#         # Update line with new Y values
#         line.set_ydata(ys)
#         line2.set_ydata(y2s)
#         ax.relim()
#         ax.autoscale(enable=True,axis='y')
#         slider_xmax.on_changed(update)
#         #plt.cla()
#         return line,line2,

#     # Set up plot to call animate() function periodically
#     ani = animation.FuncAnimation(fig, animate, interval=1, blit=True)
    
#     plt.show()


In [10]:
def check_states(checkbox_states):
    for i, var in enumerate(checkbox_states):
        if var.get() == 1:
            print(f"Checkbox {i+1} is checked")
        else:
            print(f"Checkbox {i+1} is unchecked")

In [11]:
def create_frame():
    port_text = StringVar(value="Ports")
    new_port_frame = CTkFrame(main_canvas, width = 200, height = 300, fg_color="grey", bg_color="grey")
    new_port_frame.place(x=10,y=50)
    new_port_frame.pack_propagate(0)

    PORTS = serial_ports()

    checkbox_states = []
    for i in range(len(PORTS)):
        var = IntVar()
        checkbox = CTkCheckBox(new_port_frame, text=PORTS[i], variable=var)
        checkbox.pack()
        checkbox_states.append(var)    

    # # Create a button to check the state of the checkboxes
    check_button = CTkButton(new_port_frame, text="Choose Ports", hover_color="green", 
                             command=lambda:check_states(checkbox_states))
    check_button.place(in_ = new_port_frame, x = 30, y = 260)
    refresh_button = CTkButton(new_port_frame, text="Refresh Ports", hover_color="green", 
                             command=lambda:refresh_ports(new_port_frame, check_button, refresh_button))
    refresh_button.place(in_ = new_port_frame, x = 30, y = 220)
    print("CREATED!")
    return(checkbox_states)

In [12]:
def refresh_ports(new_port_frame, check_button, refresh_button):
    check_button.destroy()
    refresh_button.destroy()
    for widgets in new_port_frame.winfo_children():
        widgets.destroy()
    return(create_frame())

In [13]:
root = CTk()
root.title("Oqni Project")
root.geometry('500x500+0+0')
root.minsize(500, 500)
root.maxsize(500, 500)

root.update()

image=Image.open('/Users/armenmkrtumyan/Desktop/oqni/robot3.png')
img=image.resize((500, 500))
bg = ImageTk.PhotoImage(img)
main_canvas = tkinter.Canvas(root, width=500, height = 500)
main_canvas.pack(fill="both", expand=True)
main_canvas.create_image(0,0,image=bg,anchor="nw")
main_canvas.image=bg

checkbox_states = create_frame()
port_text = StringVar(value="Ports")
label = CTkLabel(master=main_canvas, textvariable=port_text, width=120, height=25, corner_radius=0, text_color="black", fg_color="grey")
label.place(x = 110, y = 20, anchor=CENTER)

port_variable = StringVar()


graph_button = CTkButton(main_canvas, text = "Open graph", border_width=2, border_color="black", hover_color="green",
                         command = lambda: open_graph(port_variable.get()))
graph_button.place(in_ = main_canvas, x = 30, y = 400, anchor="sw")

testing_button = CTkButton(main_canvas, text = "Testing", border_width=2, border_color="black", hover_color="green",
                           command = lambda: testing())
testing_button.place(in_ = main_canvas, x = 30, y = 450, anchor="sw")

root.update()
root.mainloop()

CREATED!


/var/folders/k7/jqj3r_t93c5gnv6sq2gsn6mm0000gn/T/ipykernel_52287/2335950200.py:48: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani = animation.FuncAnimation(fig, animate, interval=1, blit=True)


.!ctktoplevel.!ctklabel8


/Users/armenmkrtumyan/anaconda3/envs/calc-env/lib/python3.10/site-packages/customtkinter/windows/widgets/core_widget_classes/ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")
invalid command name "5749204160update"
    while executing
"5749204160update"
    ("after" script)
invalid command name "5473166464check_dpi_scaling"
    while executing
"5473166464check_dpi_scaling"
    ("after" script)


# Creating a CSV file from sensor inputs

In [ ]:
# def create_CSV():
#     PORTS = serial_ports()
#     for port_name in PORTS:
#         OUTPUTS =[]
#         if(port_name[9] == 'u'):
#             ser = serial.Serial(port = port_name, baudrate = 9600, timeout = 0)
#             with serial.Serial(port_name, baudrate=9600) as ser:
#                 for i in range(100):
#                     s = ser.read(4)        # read up to ten bytes (timeout)
#                     OUTPUTS.append(int.from_bytes(s[:-1], "little"))
#         df = pd.DataFrame(OUTPUTS, columns=[port_name])
#     df.to_csv('~/Desktop/oqni/PORTS.csv', index=False)
#     info_clicked("Succesfully Created the CSV file", info_frame)